Calculate metric for sucess 

In [57]:
import pandas as pd
import glob
import os
from pathlib import Path

Let's explore the interactions, we already know that items and user are just information or tables which give me characteristics of the apps and type of 
linker.

In [192]:
interactions_csv_path = os.path.join(os.getcwd(), "..", "dataset", "interactions/")

all_files = glob.glob(os.path.join(interactions_csv_path, "*.csv"))

interactions_linkers_apps = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [59]:
interactions_linkers_apps["ITEM_ID"].unique()

array(['VIMEO', 'TIKTOK', 'CONTACT_DETAILS', 'PAYPAL', 'VIDEO_YOUTUBE',
       'TWITTER', 'HEADER', 'REQUEST', 'TYPEFORM', 'PODCAST', 'FACEBOOK',
       'SOUNDCLOUD', 'INSTAGRAM', 'FORM', 'PRODUCT', 'EVENTBRITE', 'BOOK',
       'PINTEREST', 'MESSAGING', 'SPOTIFY', 'DISCORD', 'DOCUMENT',
       'MUSIC', 'SQUARE', 'CALENDLY', 'SHOPIFY', 'ONLYFANS',
       'COMMERCE_PAY', 'ART', 'GO_FUND_ME', 'OPENSEA_COLLECTION',
       'SNAPCHAT', 'TWITCH', 'MEDIUM', 'MAILCHIMP', 'MOBILE_APP', 'GLEAM',
       'SPRING', 'BANDS_IN_TOWN', 'VIDEO', 'CLUBHOUSE', 'YOUTUBE',
       'REDDIT', 'COMMUNITY', 'RSS_FEED', 'NFT_GALLERY', 'AUDIOMACK',
       'CAMEO', 'PODBEAN', 'PEARPOP_PROFILE', 'TEXT_AND_MEDIA',
       'REGISTER_ACTION', '\\\\N', 'AMAZON_BOOK'], dtype=object)

There are 54 unique apps being used by linkers. Let's check how many unique apps we have registered inside items. 

In [60]:
items_apps_path = os.path.join(os.getcwd(), "..", "dataset", "items/")

all_files = glob.glob(os.path.join(items_apps_path, "*.csv"))

items_apps = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [61]:
len(items_apps["ITEM_ID"].unique())

53

So we have only 53 apps against 54 which means app \\\\N  is noise in our data or some bug, let's check some rows. 

In [62]:
interactions_linkers_apps[interactions_linkers_apps["ITEM_ID"] == "\\\\N"]

,USER_ID,ITEM_ID,TIMESTAMP
31625159,4fae5372-8d3f-4034-ac1f-5a3236252d81,\\N,1658114849
31628625,cb9163c7-b596-4a12-911f-c48b34c0046c,\\N,1657740010
31631406,a454726c-8546-4084-96fb-72162a7c93b6,\\N,1657923558
31732766,14ce4b45-6ae3-44ad-b128-24d8e6eee71c,\\N,1658179242
31733137,7716a580-6110-4b35-8830-65e065acabd1,\\N,1657725617
...,...,...,...
32771255,f4cccb8b-7a65-4065-932b-372e2f53149b,\\N,1657549407
32771965,1448eb9e-ebf6-450b-b9f8-c5872f28f48b,\\N,1657635162
32772267,0e22254a-f9e2-495b-9ef5-4ef67176d644,\\N,1657264782
32772390,9da255c9-8ede-4c0b-88f9-5f92ecde81ab,\\N,1657498894


For now I will ignore this item id because is not so many rows, then I will ask Data team about this item id later, just to clarify if it is a bug. 

In [63]:
interactions_linkers_apps_cleaned = interactions_linkers_apps[interactions_linkers_apps["ITEM_ID"] != "\\\\N"]

Let's explore distribution of apps per interactions


In [64]:
interactions_linkers_apps_cleaned_grouped = (
    interactions_linkers_apps_cleaned.groupby(["ITEM_ID"]).size().sort_values(ascending=False).reset_index(name="count")
)
interactions_linkers_apps_cleaned_grouped

,ITEM_ID,count
0,INSTAGRAM,2760600
1,DOCUMENT,2689564
2,MESSAGING,2235481
3,VIDEO_YOUTUBE,2017101
4,VIDEO,1999032
5,PRODUCT,1910128
6,YOUTUBE,1781707
7,FACEBOOK,1671282
8,MUSIC,1578335
9,HEADER,1464013


Now let's see how many times a user use an app, then we will see the median usage of a linker per app.

In [67]:
interactions_linkers_apps_median_usage = (
    interactions_linkers_apps_cleaned.groupby(["ITEM_ID", "USER_ID"])
    .size()
    .sort_values(ascending=False)
    .reset_index(name="count_per_guest")
    .groupby(["ITEM_ID"])["count_per_guest"]
    .median()
    .reset_index(name="median_usage_per_app")
)
interactions_linkers_apps_median_usage["median_usage_per_app"].median()

1.0

In Median people use the apps just once per https://linktr.ee/ space. Still we have some outliers Linkers which use some apps often. I don't have information about 
Linker deleting and putting back the same app, so I will just work with the data that is available.

In [56]:
(
    interactions_linkers_apps_cleaned.groupby(["ITEM_ID", "USER_ID"])
    .size()
    .sort_values(ascending=False)
    .reset_index(name="count_per_guest")
)

,ITEM_ID,USER_ID,count_per_guest
0,TYPEFORM,301d03c3-a474-4d2b-a977-c0071d404595,2517
1,MESSAGING,b11455d6-8f19-48be-8ca0-1e0092ac9b90,2249
2,MESSAGING,039eaa8e-0c6b-4160-a03d-79442df820bd,2186
3,DOCUMENT,a48fe532-e70b-44d0-aed2-04381def156c,1743
4,VIDEO_YOUTUBE,4a583dd9-03b6-4a31-a05f-e69f7a586023,1117
...,...,...,...
16731544,MESSAGING,54e388de-94d4-418d-93ea-d7005b699f2f,1
16731545,MESSAGING,54e3906c-dce3-47a7-8ccc-b9f19e57ea11,1
16731546,MESSAGING,54e39168-0049-453c-bcc4-3a76cb7dd466,1
16731547,MESSAGING,54e3c3f2-3db3-44f7-a215-39e185b6f725,1


Now we need to investigate how many unique apps each Linker use:

In [76]:
interactions_linkers_apps_count = (
    interactions_linkers_apps_cleaned.groupby(["USER_ID", "ITEM_ID"])
    .size()
    .sort_values(ascending=False)
    .reset_index(name="count_per_guest")
    .groupby(["USER_ID"])["count_per_guest"]
    .count()
    .reset_index(name="count_per_user_app")
)
interactions_linkers_apps_count["count_per_user_app"].median()

2.0

In [77]:
interactions_linkers_apps_count["count_per_user_app"].max()

34

So we have in median 2 apps per user in their Linktree space. We are near to our metric of success we will need to recommend apps to increase the number of apps from 2 to X. But this is considering Linkers with interactions, we need to observe Linkers outside the interactions table. 

In [194]:
from datetime import datetime
import numpy as np

linkers_csv_path = os.path.join(os.getcwd(), "..", "dataset", "users/")

all_files = glob.glob(os.path.join(linkers_csv_path, "*.csv"))

linkers = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
linkers_cleaned = linkers[linkers["TIMESTAMP"] != "\\\\N"].copy(deep=True)
linkers_cleaned["user_created_date"] = (
    linkers_cleaned["TIMESTAMP"].map(lambda date: datetime.fromtimestamp(int(date))).copy(deep=True)
)
linkers_cleaned["VERTICALS"] = (
    linkers_cleaned["VERTICALS"]
    .map(lambda verticals: verticals.split("|") if verticals is not np.nan else [])
    .copy(deep=True)
)
del linkers_cleaned["TIMESTAMP"]
linkers_cleaned

/var/folders/7k/knsjsm4903l2wbclxv90d2gh0000gn/T/ipykernel_72131/3840508500.py:7: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  linkers = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)


,USER_ID,VERTICALS,user_created_date
0,219bc28b-bc03-44d5-86d1-0cbd5620a586,"[business, fashion-&-beauty]",2021-09-11 22:45:53
1,ca930e4f-6924-4fa4-ba80-413fa9a43353,"[creative, designer]",2022-07-15 20:03:20
2,17211aa9-a4c0-487b-8b36-0efe01a9af56,"[other, personal]",2022-07-19 19:13:51
3,6463cc3c-c43c-47ad-9432-5dfda55016cf,"[education, schools-&-universities]",2022-07-18 01:08:17
4,224a9204-3021-419e-bb4a-b24969fea15f,[business],2022-07-19 20:13:34
...,...,...,...
6034802,26678a3b-4759-47ac-944b-6380b18836eb,"[creative, visual-arts]",2022-07-06 06:53:23
6034803,5903774f-2b2d-4a00-8d60-7d6b2741b605,"[business, shopping-&-retail]",2022-04-21 20:21:28
6034804,ebcc7135-bc9a-47ed-ac4e-cbc07eadb43b,"[entertainment, creative, education, non-profit]",2020-07-28 03:06:36
6034805,af1931f5-e356-4ba8-aa37-24873010788e,[business],2022-01-15 21:06:46


In [85]:
len(list(set(linkers["USER_ID"]) - set(interactions_linkers_apps_cleaned["USER_ID"])))

280

In [86]:
len(list(set(linkers["USER_ID"])))

6018887

There are only 280 users which have not interacted with apps. For these users as generic we can recommend the top 6 most used apps, or if they have type of vertical,
we can recommend the top 6 apps on their verticals. Accordingly to the data available there is not a huge impact to implement a recommender system for only new users.

Let's separate VERTICALS vs linker to produce one row per VERTICALS and linker


In [117]:
linkers_cleaned_per_vertical = linkers_cleaned.explode("VERTICALS")
linkers_cleaned_per_vertical

,USER_ID,TIMESTAMP,VERTICALS
0,219bc28b-bc03-44d5-86d1-0cbd5620a586,2021-09-11 22:45:53,business
0,219bc28b-bc03-44d5-86d1-0cbd5620a586,2021-09-11 22:45:53,fashion-&-beauty
1,ca930e4f-6924-4fa4-ba80-413fa9a43353,2022-07-15 20:03:20,creative
1,ca930e4f-6924-4fa4-ba80-413fa9a43353,2022-07-15 20:03:20,designer
2,17211aa9-a4c0-487b-8b36-0efe01a9af56,2022-07-19 19:13:51,other
...,...,...,...
6034804,ebcc7135-bc9a-47ed-ac4e-cbc07eadb43b,2020-07-28 03:06:36,education
6034804,ebcc7135-bc9a-47ed-ac4e-cbc07eadb43b,2020-07-28 03:06:36,non-profit
6034805,af1931f5-e356-4ba8-aa37-24873010788e,2022-01-15 21:06:46,business
6034806,c2df1556-cf00-4f4e-ab24-af6027be0d49,2021-11-25 05:06:24,travel-&-tourism


In [118]:
linkers_cleaned_per_vertical_grouped = (
    linkers_cleaned_per_vertical.groupby(["VERTICALS"]).size().sort_values(ascending=False).reset_index(name="count")
)
linkers_cleaned_per_vertical_grouped

,VERTICALS,count
0,business,1774667
1,creative,1357287
2,entertainment,1330416
3,music,597580
4,other,567455
...,...,...
107,museums,1219
108,electronic-press-kit,891
109,military-&-veterans,730
110,library,615


In [135]:
interaction_per_vertical_linker = interactions_linkers_apps_cleaned.merge(
    linkers_cleaned, how="left", on="USER_ID"
).explode("VERTICALS")
interaction_per_vertical_linker

,USER_ID,ITEM_ID,TIMESTAMP,VERTICALS,user_created_date
0,bfa53e97-4d1b-45ed-b7f6-1a499a26ed34,VIMEO,1658235377,business,2022-05-05 19:02:37
1,5888da2a-a642-437f-bc53-a341a028eeb8,TIKTOK,1658236059,creative,2022-05-21 15:53:38
1,5888da2a-a642-437f-bc53-a341a028eeb8,TIKTOK,1658236059,influencer,2022-05-21 15:53:38
2,c0a0958f-aee0-4864-8bb2-8e923cd8c494,CONTACT_DETAILS,1658236018,business,2022-07-04 11:39:37
3,5e6417a3-31e1-4c6d-aff6-f5b47ce443f2,PAYPAL,1658235487,creative,2022-02-28 23:05:38
...,...,...,...,...,...
32899081,0df94708-82ee-44a4-b9d7-4baa52373bcf,MAILCHIMP,1657228922,coffee-&-tea,2022-07-08 07:20:14
32899082,78703518-057d-48bd-949b-2e8c9b6aa820,ONLYFANS,1657393785,creative,2022-01-06 20:53:49
32899082,78703518-057d-48bd-949b-2e8c9b6aa820,ONLYFANS,1657393785,adult,2022-01-06 20:53:49
32899083,ea5c43e6-ca7e-4ba2-b3d6-cf0414c10c33,RSS_FEED,1657500859,other,2022-05-25 13:54:05


Let's get how every vertical is interacting with apps

In [132]:
interaction_per_vertical_linker_grouped = (
    interaction_per_vertical_linker.groupby(["VERTICALS"]).size().sort_values(ascending=False).reset_index(name="count")
)
interaction_per_vertical_linker_grouped

,VERTICALS,count
0,entertainment,8892222
1,business,8721906
2,creative,7572341
3,music,4559947
4,other,2904055
...,...,...
107,law-enforcement,5485
108,electronic-press-kit,4765
109,military-&-veterans,4270
110,library,3887


In [134]:
interaction_per_vertical_linker_count = (
    interaction_per_vertical_linker.groupby(["VERTICALS", "ITEM_ID"])
    .size()
    .sort_values(ascending=False)
    .reset_index(name="count_per_vertical")
    .groupby(["VERTICALS"])["count_per_vertical"]
    .count()
    .reset_index(name="count_per_vertical_app")
)
interaction_per_vertical_linker_count["count_per_vertical_app"].median()

48.0

Verticals and interaction count are correlated, this means we have more Linkers in entertainment and then we have more interaction with apps from these vertical. 
On median verticals have 48 apps interactions.

We would like to know accordingly to Linker creation date, if a Linker in one vertical have more interactions with other apps because is old in the system, 
and how we can compare these to same Linkers in the same vertical but with newer creation date.

In [136]:
interaction_per_vertical_linker_entertainment = interaction_per_vertical_linker[
    interaction_per_vertical_linker["VERTICALS"] == "entertainment"
]
interaction_per_vertical_linker_entertainment

In [140]:
interaction_per_vertical_linker_entertainment.groupby(["USER_ID", "ITEM_ID"])["user_created_date"].size().sort_values(
    ascending=False
).reset_index(name="count_per_guest")

,USER_ID,ITEM_ID,count_per_guest
0,d3bbf3f6-d451-4d77-aeb6-bd778f4c277a,VIDEO,938
1,d3bbf3f6-d451-4d77-aeb6-bd778f4c277a,YOUTUBE,938
2,dd020b35-5ce6-4587-b4b9-0c6f25a03cd4,MUSIC,932
3,997ab038-c5b3-49e8-8417-6dbcd8699607,SPOTIFY,850
4,997ab038-c5b3-49e8-8417-6dbcd8699607,MUSIC,850
...,...,...,...
4518418,320c8ae2-3b62-4940-81df-9133322210de,TWITCH,1
4518419,320c8ae2-3b62-4940-81df-9133322210de,INSTAGRAM,1
4518420,320c6880-7fd5-49a5-9590-40cc8cdbc0bb,SOUNDCLOUD,1
4518421,320c6210-14fb-41f3-a41f-2d91d6c00ea5,VIDEO_YOUTUBE,1


In [158]:
interaction_per_vertical_linker_entertainment_count = (
    interaction_per_vertical_linker_entertainment.groupby(["USER_ID", "ITEM_ID"])
    .size()
    .sort_values(ascending=False)
    .reset_index(name="count_per_guest")
    .groupby(["USER_ID"])["count_per_guest"]
    .count()
    .reset_index(name="count_per_user_app")
)
interaction_per_vertical_linker_entertainment_count

interaction_linker_entertainment_count_with_creation_date = interaction_per_vertical_linker_entertainment_count.merge(
    linkers_cleaned[["USER_ID", "user_created_date"]], how="left", on="USER_ID"
).sort_values(by="user_created_date", ascending=False)
interaction_linker_entertainment_count_with_creation_date

,USER_ID,count_per_user_app,user_created_date
762419,92ca9646-50ce-4d83-b6a0-f4c8f358c605,1,2022-07-20 10:13:32
105917,145ff7b1-2ace-46b0-8b97-9c455ac5c09b,2,2022-07-20 10:11:25
392839,4b91ef85-afe4-4599-8f45-e5f3cfa739f9,1,2022-07-20 10:09:37
163862,1f9b750b-c859-4311-83ea-d3e5e703bd4c,2,2022-07-20 10:08:55
503396,60f5a4d1-5c93-45c1-904f-17e0aec7365d,2,2022-07-20 10:07:28
...,...,...,...
1155708,de5c69b0-1656-47f5-994e-c49ccec9bc40,2,2019-11-08 03:48:22
756737,91aef8a2-ac3a-4350-b602-4cf618302e66,2,2019-11-07 22:41:31
822494,9e64c20d-aae0-49e5-9ba9-454e35b3c60e,8,2019-11-07 21:48:00
13129,028cd51b-ea38-4394-9951-3e73f2da158c,1,2019-11-07 21:26:11


In [174]:
interaction_linker_entertainment_count_with_creation_date[
    interaction_linker_entertainment_count_with_creation_date["user_created_date"] < "2021-01-01"
]["count_per_user_app"].median()

3.0

In [172]:
interaction_linker_entertainment_count_with_creation_date[
    interaction_linker_entertainment_count_with_creation_date["user_created_date"] > "2022-01-01"
]["count_per_user_app"].median()

3.0

It seems date does not affect the median interaction apps per Linker. Linkers register before 2021 still use 3 apps interactions in median the same as Linker registered after January 2022. 

In other hand we can get kind of the most experience Linkers, this means Linkers that are inside
the percentile 95 using the count per apps on users, these linkers will have the higher apps interaction count, and can be a good base to compare to to Linkers that perform with 2-3 apps interactions.

In [177]:
print("quantile 0.95", interaction_linker_entertainment_count_with_creation_date["count_per_user_app"].quantile(0.95))
print("quantile 0.7", interaction_linker_entertainment_count_with_creation_date["count_per_user_app"].quantile(0.7))
print("quantile 0.5", interaction_linker_entertainment_count_with_creation_date["count_per_user_app"].quantile(0.5))
interaction_linker_entertainment_count_with_creation_date[
    interaction_linker_entertainment_count_with_creation_date["count_per_user_app"] >= 8
]

quantile 0.95 8.0
quantile 0.7 4.0
quantile 0.5 3.0


,USER_ID,count_per_user_app,user_created_date
745676,8f902b7e-0aff-4bca-bc3a-d32a06d2b2fb,9,2022-07-20 08:07:19
221200,2a9720ea-e36d-408b-a17e-76b9d4fb2fc2,8,2022-07-20 08:02:59
494476,5f3d4565-8143-4e74-85dc-18b961a100c7,8,2022-07-20 07:30:30
332866,4014fae8-3f8e-4abb-a91d-717b0aa8fa62,10,2022-07-20 07:02:47
332865,4014fae8-3f8e-4abb-a91d-717b0aa8fa62,10,2022-07-20 07:02:47
...,...,...,...
864310,a66a0836-1e55-48b1-883d-361bb1273f76,9,2019-11-12 00:42:11
1086749,d1211ebc-76d7-475b-87f3-d4d5e9ba5191,8,2019-11-09 06:53:23
644328,7c1ea31b-a2d0-4f3a-b79d-77049a0e34ab,11,2019-11-09 04:12:24
775952,9562a112-e6bf-4551-9f4c-a17672d11061,9,2019-11-08 09:13:13


In [178]:
interaction_linker_entertainment_count_with_creation_date[
    interaction_linker_entertainment_count_with_creation_date["count_per_user_app"] <= 3
]

,USER_ID,count_per_user_app,user_created_date
762419,92ca9646-50ce-4d83-b6a0-f4c8f358c605,1,2022-07-20 10:13:32
105917,145ff7b1-2ace-46b0-8b97-9c455ac5c09b,2,2022-07-20 10:11:25
392839,4b91ef85-afe4-4599-8f45-e5f3cfa739f9,1,2022-07-20 10:09:37
163862,1f9b750b-c859-4311-83ea-d3e5e703bd4c,2,2022-07-20 10:08:55
503396,60f5a4d1-5c93-45c1-904f-17e0aec7365d,2,2022-07-20 10:07:28
...,...,...,...
652111,7d99b6d0-250c-438d-94ce-1e6a0b9cedd4,2,2019-11-08 04:01:29
1155708,de5c69b0-1656-47f5-994e-c49ccec9bc40,2,2019-11-08 03:48:22
756737,91aef8a2-ac3a-4350-b602-4cf618302e66,2,2019-11-07 22:41:31
13129,028cd51b-ea38-4394-9951-3e73f2da158c,1,2019-11-07 21:26:11


In [190]:
experienced_linker = "7c1ea31b-a2d0-4f3a-b79d-77049a0e34ab"
non_experienced_linker = "92ca9646-50ce-4d83-b6a0-f4c8f358c605"
non_experienced_linker2 = "28bc05d0-e8ad-462d-ab20-8dbc74dc25cd"
non_experienced_linker3 = "7d99b6d0-250c-438d-94ce-1e6a0b9cedd4"

In [182]:
interactions_linkers_apps_cleaned[interactions_linkers_apps_cleaned["USER_ID"] == experienced_linker].drop_duplicates(
    subset=["ITEM_ID"]
)

,USER_ID,ITEM_ID,TIMESTAMP
1110734,7c1ea31b-a2d0-4f3a-b79d-77049a0e34ab,COMMERCE_PAY,1657089559
2224972,7c1ea31b-a2d0-4f3a-b79d-77049a0e34ab,SQUARE,1655771622
3072517,7c1ea31b-a2d0-4f3a-b79d-77049a0e34ab,TWITTER,1655771628
5004951,7c1ea31b-a2d0-4f3a-b79d-77049a0e34ab,INSTAGRAM,1657089706
10983738,7c1ea31b-a2d0-4f3a-b79d-77049a0e34ab,FACEBOOK,1657089588
13140480,7c1ea31b-a2d0-4f3a-b79d-77049a0e34ab,REQUEST,1655771617
13513470,7c1ea31b-a2d0-4f3a-b79d-77049a0e34ab,VIDEO_YOUTUBE,1655771627
18941961,7c1ea31b-a2d0-4f3a-b79d-77049a0e34ab,PAYPAL,1655771617
20371365,7c1ea31b-a2d0-4f3a-b79d-77049a0e34ab,TWITCH,1657089722
21727455,7c1ea31b-a2d0-4f3a-b79d-77049a0e34ab,VIDEO,1657089722


In [183]:
interactions_linkers_apps_cleaned[
    interactions_linkers_apps_cleaned["USER_ID"] == non_experienced_linker
].drop_duplicates(subset=["ITEM_ID"])

,USER_ID,ITEM_ID,TIMESTAMP
9393,92ca9646-50ce-4d83-b6a0-f4c8f358c605,SPRING,1658277337


In [191]:
interactions_linkers_apps_cleaned[
    interactions_linkers_apps_cleaned["USER_ID"] == non_experienced_linker2
].drop_duplicates(subset=["ITEM_ID"])

,USER_ID,ITEM_ID,TIMESTAMP
21656059,28bc05d0-e8ad-462d-ab20-8dbc74dc25cd,VIDEO,1641021805
25578397,28bc05d0-e8ad-462d-ab20-8dbc74dc25cd,YOUTUBE,1641021805


In [189]:
interactions_linkers_apps_cleaned[
    interactions_linkers_apps_cleaned["USER_ID"] == non_experienced_linker3
].drop_duplicates(subset=["ITEM_ID"])

,USER_ID,ITEM_ID,TIMESTAMP
667392,7d99b6d0-250c-438d-94ce-1e6a0b9cedd4,MUSIC,1649648697
28021095,7d99b6d0-250c-438d-94ce-1e6a0b9cedd4,TIKTOK,1649648513


So concluding this hypothesis, we can see the experience Linker have related apps than comparing to the another non experience Linkers. Some of the apps are related to video 
and to purchases. Also as we commented before we wanted to explore how much increase apps interaction we could increase per Linker, in this case using verticals and 
experienced user we can make an estimation of grow. In the case of entertainment vertical we have an estimation from 3 interaction to 8 interaction. 
Our recommender system should help as in the example each vertical, and using this information our recommender system in estimation should recommend apps from
experienced users, after this recommendation is done, we estimate to increase the median of 3 apps but not more than 8 apps. 